##Importing Libraries


In [0]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt
# from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn import metrics
from sklearn.metrics import log_loss
train_df = pd.read_json("train.json")
test_df = pd.read_json("test.json")

##Training Data

In [2]:
# train_df["created"] = pd.to_datetime(train_df["created"])
# train_df["created_year"] = train_df["created"].dt.year
# train_df["created_month"] = train_df["created"].dt.month
# train_df["created_day"] = train_df["created"].dt.day
# train_df["num_features"] = train_df["features"].apply(len)
# train_df["num_photos"] = train_df["photos"].apply(len)
train_df.drop(columns=['created', 'display_address', 'description', 'street_address', 'photos'], inplace=True)
train_df.drop(columns=['building_id', 'manager_id', 'listing_id'], inplace=True)
train_df.drop(columns=['features'], inplace=True)
train_df

,bathrooms,bedrooms,latitude,longitude,price,interest_level
4,1.0,1,40.7108,-73.9539,2400,medium
6,1.0,2,40.7513,-73.9722,3800,low
9,1.0,2,40.7575,-73.9625,3495,medium
10,1.5,3,40.7145,-73.9425,3000,medium
15,1.0,0,40.7439,-73.9743,2795,low
...,...,...,...,...,...,...
124000,1.0,3,40.8433,-73.9396,2800,low
124002,1.0,2,40.8198,-73.9578,2395,medium
124004,1.0,1,40.5765,-73.9554,1850,medium
124008,1.0,2,40.7448,-74.0017,4195,medium


In [0]:
# train_df['interest_level'].value_counts()

##Testing Data

In [4]:
test_df2 = test_df.copy()
# test_df2["created"] = pd.to_datetime(test_df2["created"])
# test_df2["created_year"] = test_df2["created"].dt.year
# test_df2["created_month"] = test_df2["created"].dt.month
# test_df2["created_day"] = test_df2["created"].dt.day
# test_df2["num_features"] = test_df2["features"].apply(len)
# # test_df2["num_photos"] = test_df2["photos"].apply(len)
test_df2.drop(columns=['created', 'display_address', 'description', 'street_address', 'photos'], inplace=True)
test_df2.drop(columns=['building_id', 'manager_id', 'listing_id'], inplace=True)
test_df2.drop(columns=['features'], inplace=True)
test_df2

,bathrooms,bedrooms,latitude,longitude,price
0,1.0,1,40.7185,-73.9865,2950
1,1.0,2,40.7278,-74.0000,2850
2,1.0,0,40.7260,-74.0026,2295
3,1.0,2,40.7321,-74.0028,2900
5,1.0,1,40.7054,-74.0095,3254
...,...,...,...,...,...
124003,1.0,1,40.7925,-73.9454,1700
124005,1.0,2,40.7456,-73.9797,4195
124006,1.0,0,40.7416,-73.9829,2400
124007,2.0,2,40.7485,-73.9800,6895


In [0]:
# train_df["features"].apply(len)

In [0]:
# test_df["features"].apply(len)

##Logistic Regression Classifier

In [0]:
target_map = {'high':0, 'medium':1, 'low':2}
X = train_df[train_df.columns.difference(['interest_level'])]
y = np.array(train_df['interest_level'].apply(lambda x: target_map[x]))
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33)
lr = LogisticRegressionCV(cv=5, random_state=0)
lr.fit(X_train, y_train)
y_val_pred = lr.predict_proba(X_val)
y_val_pred_acc = lr.predict(X_val)

In [8]:
logloss = metrics.log_loss(y_val, y_val_pred)
logloss

0.7480296129997361

In [9]:
accuracy = metrics.accuracy_score(y_val, y_val_pred_acc)
accuracy

0.6894455700865721

##Precision, Recall, F1-Score



In [10]:
#metrics.confusion_matrix(y_val, y_val_pred_acc)
metrics.classification_report(y_val, y_val_pred_acc)

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


'              precision    recall  f1-score   support\n\n           0       0.00      0.00      0.00      1320\n           1       0.00      0.00      0.00      3737\n           2       0.69      1.00      0.82     11230\n\n    accuracy                           0.69     16287\n   macro avg       0.23      0.33      0.27     16287\nweighted avg       0.48      0.69      0.56     16287\n'

##Submission File

In [0]:
X_test = test_df2
y_test = lr.predict_proba(X_test)
sub = pd.DataFrame()
sub['listing_id'] = test_df['listing_id']
for label in ['high', 'medium', 'low']:
  sub[label] = y_test[:, target_map[label]]
sub.to_csv("submission.csv", index=False)